In [34]:
# === Hücre 1: Kütüphane Yükleme ve Yol Kontrolü ===

from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm

# Proje kökünü tanımla (senin repo dizinin)
PROJECT_ROOT = Path("/Users/berkaybakac/Projects/geosentinel")

# Ham veri yolu
RAW_DATA_PATH = PROJECT_ROOT / "data" / "raw"

# Geolife dataset kökü
GEOLIFE_PATH = RAW_DATA_PATH / "geolife"
DATA_DIR = GEOLIFE_PATH / "Data"

# Yol kontrolü
if not DATA_DIR.exists():
    raise FileNotFoundError(f"Veri dizini bulunamadı: {DATA_DIR}")
else:
    print(f"✓ Veri dizini bulundu: {DATA_DIR}")

# Küçük sağlık kontrolü (opsiyonel ama çok faydalı)
user_folders = [d for d in DATA_DIR.iterdir() if d.is_dir()]
plt_count = sum(len(list((u / "Trajectory").glob("*.plt"))) for u in user_folders)
print(f"✓ Kullanıcı klasörü sayısı: {len(user_folders)} (≈181 beklenir)")
print(f"✓ Toplam .plt dosyası   : {plt_count} (≈18k beklenir)")

# Örnek dosya göstermek
if user_folders:
    sample_user = user_folders[0]
    sample_file = next((sample_user / "Trajectory").glob("*.plt"))
    print(f"✓ Örnek dosya: {sample_file}")


✓ Veri dizini bulundu: /Users/berkaybakac/Projects/geosentinel/data/raw/geolife/Data
✓ Kullanıcı klasörü sayısı: 182 (≈181 beklenir)
✓ Toplam .plt dosyası   : 18670 (≈18k beklenir)
✓ Örnek dosya: /Users/berkaybakac/Projects/geosentinel/data/raw/geolife/Data/135/Trajectory/20090103012134.plt


In [35]:
# === Hücre 2: Fonksiyon Tanımları ===

def read_plt_file(fp: Path) -> pd.DataFrame:
    """Tek .plt dosyasını oku → DataFrame."""
    df = pd.read_csv(
        fp, skiprows=6, header=None,
        names=["lat","lon","ignore","alt_ft","days","date","time"],
        encoding="utf-8", on_bad_lines="skip"
    )
    if df.empty:
        return df

    # Zaman
    ts = pd.to_datetime(df["date"] + " " + df["time"], errors="coerce")
    df = df.loc[ts.notna()].copy()
    df["timestamp"] = ts[ts.notna()]

    # Minimal kolon seti
    out = pd.DataFrame({
        "user_id":  fp.parent.parent.name,  # Data/<user>/Trajectory/<file>
        "traj_id":  fp.stem,                # dosya adı
        "timestamp": df["timestamp"],
        "lat":      df["lat"].astype("float32"),
        "lon":      df["lon"].astype("float32"),
        "alt_ft":   df["alt_ft"].astype("float32"),
    })
    return out

def load_all_geolife_data(data_dir: Path,
                          limit_users: int | None = None,
                          limit_files: int | None = None) -> pd.DataFrame:
    """Tüm kullanıcıların .plt dosyalarını oku → birleşik DataFrame."""
    users = [d for d in sorted(data_dir.iterdir()) if d.is_dir()]

    if limit_users is not None:
        users = users[:limit_users]

    chunks: list[pd.DataFrame] = []

    for u in tqdm(users, desc="Kullanıcılar"):
        traj_dir = u / "Trajectory"
        if not traj_dir.exists():
            continue

        files = sorted(traj_dir.glob("*.plt"))

        if limit_files is not None:
            files = files[:limit_files]

        for f in files:
            df = read_plt_file(f)
            if not df.empty:
                chunks.append(df)

    if not chunks:
        return pd.DataFrame()

    return pd.concat(chunks, ignore_index=True)


In [36]:
# === Hücre 3: Fonksiyon Testi (küçük örnek) ===
test_df = load_all_geolife_data(DATA_DIR, limit_users=2, limit_files=3)
print("Satır:", len(test_df))
test_df.head()

Kullanıcılar: 100%|██████████| 2/2 [00:00<00:00, 47.48it/s]

Satır: 12061


,user_id,traj_id,timestamp,lat,lon,alt_ft
0,000,20081023025304,2008-10-23 02:53:04,39.984703,116.318420,492.0
1,000,20081023025304,2008-10-23 02:53:10,39.984684,116.318451,492.0
2,000,20081023025304,2008-10-23 02:53:15,39.984688,116.318420,492.0
3,000,20081023025304,2008-10-23 02:53:20,39.984688,116.318382,492.0
4,000,20081023025304,2008-10-23 02:53:25,39.984653,116.318260,492.0
